# AstrID Processing Notebook

This notebook loads sky cutouts or local FITS images, applies lightweight preprocessing, and produces QA plots. It relies on reusable helpers under `src/adapters/`.


In [6]:
# Setup
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))
print(f"Project root: {project_root}")


Project root: /home/chris/github/AstrID


In [7]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from pathlib import Path

from src.adapters.imaging.preprocess import preprocess_image
from src.adapters.imaging.utils import to_display_image
from src.adapters.external.skyview import SkyViewClient
from src.adapters.external.mast import MASTClient


In [8]:
# Helper: display side-by-side
from typing import Optional

def show_side_by_side(img_a: np.ndarray, img_b: Optional[np.ndarray] = None, titles=("input", "processed")):
    if img_b is None:
        plt.figure(figsize=(4,4))
        if img_a.ndim == 2:
            plt.imshow(img_a, origin="lower", cmap="gray")
        else:
            plt.imshow(img_a, origin="lower")
        plt.title(titles[0])
        plt.tight_layout()
        plt.show()
        return
    fig, axes = plt.subplots(1,2, figsize=(8,4))
    if img_a.ndim == 2:
        axes[0].imshow(img_a, origin="lower", cmap="gray")
    else:
        axes[0].imshow(img_a, origin="lower")
    axes[0].set_title(titles[0])
    if img_b.ndim == 2:
        axes[1].imshow(img_b, origin="lower", cmap="gray")
    else:
        axes[1].imshow(img_b, origin="lower")
    axes[1].set_title(titles[1])
    plt.tight_layout()
    plt.show()


In [9]:
# Load or fetch an example image
# Prefer SkyView via DSS; if unavailable, fall back to PS1 JPEG display
ra_deg, dec_deg = 180.0, 0.0
img, info = SkyViewClient.fetch_reference_image(ra_deg, dec_deg, size_pixels=300, fov_deg=0.02, to_display_image_fn=to_display_image)
if img is None:
    print("SkyView unavailable; falling back to PS1 JPEG display...")
    img, info = MASTClient.fetch_ps1_cutout(ra_deg, dec_deg, size_pixels=240, filt="g")

if img is not None:
    show_side_by_side(img)
else:
    print("No image available to process.")


KeyboardInterrupt: 

In [10]:
# Apply preprocessing and visualize
if img is not None:
    # Ensure 2D input for preprocess; if RGB, convert via grayscale within preprocess
    processed = preprocess_image(img, kernel_size=(3,3), threshold_value=100)
    show_side_by_side(to_display_image(img), processed)
else:
    print("Skip preprocessing; no image available.")


NameError: name 'img' is not defined

In [11]:
# Optional: cache processed image to disk (staging)
from datetime import datetime
from pathlib import Path
import json

if img is not None:
    cache_dir = Path("data/ingestion_cache/processed")
    cache_dir.mkdir(parents=True, exist_ok=True)
    ts = datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
    meta = {"ra": ra_deg, "dec": dec_deg, "source": info.get("source"), "format": info.get("format")}
    np.save(cache_dir / f"proc_{ts}.npy", processed)
    with open(cache_dir / f"proc_{ts}.json", "w") as f:
        json.dump(meta, f)
    print(f"Saved processed arrays and metadata under {cache_dir}")
else:
    print("No processed output to cache.")


NameError: name 'img' is not defined